# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298512908869                   -0.85    4.8
  2   -8.300197985305       -2.77       -1.26    1.6
  3   -8.300443789896       -3.61       -1.88    3.0
  4   -8.300460953100       -4.77       -2.71    1.4
  5   -8.300464267551       -5.48       -3.16    3.2
  6   -8.300464452347       -6.73       -3.29    9.8
  7   -8.300464535798       -7.08       -3.42    1.1
  8   -8.300464587133       -7.29       -3.54    2.4
  9   -8.300464630616       -7.36       -3.76    1.0
 10   -8.300464639564       -8.05       -3.92    3.6
 11   -8.300464642916       -8.47       -4.21    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64167321000                   -0.70    6.1
  2   -16.64598990080       -2.36       -1.14    1.0
  3   -16.67915341007       -1.48       -1.86    3.0
  4   -16.67922252102       -4.16       -2.59    5.2
  5   -16.67928094072       -4.23       -3.06    4.2
  6   -16.67928575083       -5.32       -3.41    2.9
  7   -16.67928618761       -6.36       -3.79    2.1
  8   -16.67928622348       -7.45       -4.59    1.2


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32624237995                   -0.56    7.4
  2   -33.33420272479       -2.10       -1.00    2.0
  3   -33.33595243211       -2.76       -1.72    5.2
  4   -33.33683250836       -3.06       -2.30    6.9
  5   -33.33693336783       -4.00       -2.81    9.1
  6   -33.33694387719       -4.98       -3.81    3.1
  7   -33.33694391094       -7.47       -4.11    5.4


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298146505667                   -0.85    5.8
  2   -8.300202265116       -2.69       -1.58    1.0
  3   -8.300332935644       -3.88       -2.17    6.6
  4   -8.300320500254   +   -4.91       -2.17    3.1
  5   -8.300463951934       -3.84       -3.46    1.2
  6   -8.300464567470       -6.21       -3.84    3.8
  7   -8.300464636265       -7.16       -4.28    5.0


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32722934413                   -0.56    8.4
  2   -33.32522033687   +   -2.70       -1.27    1.5
  3   -20.99215215341   +    1.09       -0.52    7.4
  4   -33.19639557516        1.09       -1.43    6.0
  5   -33.24579604853       -1.31       -1.36    4.1
  6   -32.98490335932   +   -0.58       -1.29    3.9
  7   -33.33188160520       -0.46       -2.19    5.2
  8   -33.33543859584       -2.45       -2.35    1.9
  9   -33.33596336386       -3.28       -2.51    2.0
 10   -33.33636560646       -3.40       -2.64    1.5
 11   -33.33678822119       -3.37       -2.92    2.2
 12   -33.33693971313       -3.82       -3.53    4.0
 13   -33.33694258261       -5.54       -3.68    3.2
 14   -33.33694209113   +   -6.31       -3.78    2.6
 15   -33.33694373599       -5.78       -4.29    2.0


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.